In [7]:
%matplotlib inline

from cmocean import cm
import matplotlib.pyplot as plt
import mpl_toolkits
import numpy as np

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.tri as Tri

import pandas as pd
import scipy
import matplotlib as mpl

from pylab import *
import cartopy as cart

import xarray as xr
from netCDF4 import Dataset
from dateutil import parser
from datetime import datetime,timedelta
from os import listdir
import plotly.graph_objects as go

In [8]:
def fvcom_netcdf(dataset):
    # Dropping the problematic variables
    drop_variables = ['siglay','siglev']
    ds = xr.open_dataset(dataset,drop_variables=drop_variables,decode_times=False)

    # convert lon/c, lat/c to coordinates
    ds = ds.assign_coords({var:ds[var] for var in ['lon','lat','lonc','latc']})


    #Solving the problem with siglay and siglev. We need to workaround using netCDF4 and renaming the coordinates.
    # load data with netCDF4
    nc = Dataset(dataset)
    # load the problematic coordinates
    coords = {name:nc[name] for name in drop_variables}

    # function to extract ncattrs from `Dataset()`
    get_attrs = lambda name: {attr:coords[name].getncattr(attr) for attr in coords[name].ncattrs()}
    # function to convert from `Dataset()` to `xr.DataArray()`
    nc2xr = lambda name: xr.DataArray(coords[name],attrs=get_attrs(name),name=f'{name}_coord',dims=(f'{name}','node'))

    # apply `nc2xr()` and merge `xr.DataArray()` objects
    coords = xr.merge([nc2xr(name) for name in coords.keys()])

    # reassign to the main `xr.Dataset()`
    ds = ds.assign_coords(coords)


    #We now can assign the z coordinate for the data.
    z = (ds.siglay_coord*ds.h)
    z.attrs = dict(long_name='nodal z-coordinate',units='meters')
    ds = ds.assign_coords(z=z)


    #The time coordinate still needs some fixing. We will parse it and reassign to the dataset.
    # the first day
    dt0 = parser.parse(ds.time.attrs['units'].replace('days since ',''))
    # parse dates summing days to the origin
    ds = ds.assign(time=[dt0+timedelta(seconds=day*86400) for day in ds.time.values])
    
    return(ds)

## User inputs

In [9]:
layers = [0,10,19]
time = [1,2,3,4,5,6,7,8,9]
variable = 'currents'#'currents'#'salinity'  #temp, u, v
directory = r'C:\Users\Felicio.Cassalho\skill_asess_results_sfbofs\nowcast_model_20221202' #where the data to be concatenated is stored
file = r'\nos.sfbofs.fields.n003.20221202.t15z.nc'

In [10]:
FVCOM = fvcom_netcdf(directory+file)

### Triangulation

In [5]:
x, y = FVCOM['lon'], FVCOM['lat']
triangles = FVCOM['nv'].transpose() - 1

# 3D Plots for Vectorial and Scalar Variables

In [6]:
# make figure
fig_dict = {
    "data": [],
    # "layout": {},
    "frames": []
}

if variable == 'currents':
    # make data
    for l in layers:
        data_dict = {
            "x": np.array(x),
            "y": np.array(y),
            "z": np.array(FVCOM['z'][l]),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            "intensity":np.array(((FVCOM['u'][0][l]**2)+(FVCOM['v'][0][l]**2))**.5),
            "intensitymode":"cell",
            #"colorscale":'rainbow',
            "coloraxis":"coloraxis",
            "opacity":1,
        }
        fig_dict["data"].append(data_dict)
    # make frames
    for t in time:
        frame = {"data": [], "name": str(FVCOM['Times'][t]).split()[3].split("'")[1][:-7]}   
        for l in layers:
            frame_dict = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(FVCOM['z'][l]),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                "intensity":np.array(((FVCOM['u'][t][l]**2)+(FVCOM['v'][t][l])**2)**.5),
                "intensitymode":"cell",
                #"colorscale":'rainbow',
                "coloraxis":"coloraxis",
                "cmax":1,
                "opacity":1,
            }

            frame["data"].append(frame_dict)
        fig_dict["frames"].append(frame)   
else:
    # make data
    for l in layers:
        data_dict = {
            "x": np.array(x),
            "y": np.array(y),
            "z": np.array(FVCOM['z'][l]),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            "intensity":np.array(FVCOM[variable][0][l]),
            #"intensitymode":"cell",
            #"colorscale":'rainbow',
            "coloraxis":"coloraxis",
            "opacity":.5,
        }
        fig_dict["data"].append(data_dict)    
    # make frames
    for t in time:
        frame = {"data": [], "name": str(FVCOM['Times'][t]).split()[3].split("'")[1][:-7]}   
        for l in layers:
            frame_dict = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(FVCOM['z'][l]),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                "intensity":np.array(FVCOM[variable][t][l]),
                #"intensitymode":"cell",
                #"colorscale":'rainbow',
                "coloraxis":"coloraxis",
                "opacity":.5,
            }

            frame["data"].append(frame_dict)
        fig_dict["frames"].append(frame)   

IndexError: index 1 is out of bounds for axis 0 with size 1

In [246]:
fig=go.Figure(data=[go.Mesh3d(fig_dict['data'][l]) for l in range(len(layers))],
              layout=go.Layout(xaxis=dict(range=[0, 1], autorange=False),
                               yaxis=dict(range=[0, 1], autorange=False),
                               title=variable,
                               updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",args=[None])])]),
              frames = [go.Frame(data=[go.Mesh3d(fig_dict['frames'][t]['data'][l]) for l in range(len(layers))], name = f'fr{t}') for t in range(len(time))]
             )


sliders =  [dict(steps= [dict(method= 'animate',
                              args= [[f'fr{t}'], #this steps refers to the frame of name f'fr{k}
                                      dict(mode= 'immediate',
                                           frame= dict( duration=300, redraw= True ),
                                           fromcurrent=True,
                                           transition=dict( duration= 0))],
                                      label=str(FVCOM['Times'][t]).split()[3].split("'")[1][:-7]) for t  in time], #this is the step label on the slider 
                                      minorticklen=0,
                                      x=0,
                                   len=1)]

fig.update_layout(sliders=sliders)

fig.write_html('first_figure_fvcom.html', auto_open=True)
# fig.show()